In [3]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from category_encoders import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score, mean_absolute_error
from xgboost import XGBRegressor
import pickle
import random
import shap
from pdpbox.pdp import pdp_isolate, pdp_plot
import datetime
from datetime import datetime, timedelta

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [4]:
random.seed(42)
np,random.seed(42)

(<module 'numpy' from '/Users/bumsoojoe/anaconda3/envs/tp/lib/python3.8/site-packages/numpy/__init__.py'>,
 None)

In [5]:
## 비행시간 출력 함수
def Flight_Time(Departure_or_entry,Class,Flight_type):
    # 비행 시간표
    departure = pd.DataFrame({'비즈니스': [11.335042016806725, 22.935111930471425, 25.619097639981486, 28.99587301587302, 51.64812499999999], 
             '이코노미': [11.215637583892619, 23.28843523073319, 25.919412673879446, 26.798203497615262, 51.6],
             '프리미엄 이코노미': [11.302, 23.054464520367937, 26.041043719989894, 26.864802784222736, 51], 
             '퍼스트': [11.169999999999995, 19.993137996219282, 24.588079999999998, 29.370550660792954, 51]}).round(2)
    entry = pd.DataFrame({'비즈니스': [13.326504065040652, 25.909293025470127, 28.1739769065521, 36.17582456140351, 54.69217391304348], 
         '이코노미': [13.261862068965517, 25.64626043841336, 28.733799201369084, 29.991457399103137, 56], 
         '프리미엄 이코노미': [13.166, 25.593388288800455, 28.34211437170805, 33.18008445945946, 81.385], 
         '퍼스트': [13.33435483870968, 24.71732824427481, 28.076642066420664, 31.87838331160365, 88.10833333333333]}).round(2)
    # 비행 시간 출력
    if Departure_or_entry == '출국':
        return departure[Class][Flight_type]
    elif Departure_or_entry == '귀국':
        return entry[Class][Flight_type]

## 날짜 데이터 처리
def Date_data(Date):
    dt = datetime(Date)
    day = dt.day
    # 월:0,화:1,수:2,목:3,금:4,토:5,일:6 /->전환/ '토':2, '금':1, '화':5, '수':6, '일':3, '월':4, '목':7
    week_list = {'0':'월','1':'화','2':'수','3':'목','4':'금','5':'토','6':'일'}
    week_code = {'토':2, '금':1, '화':5, '수':6, '일':3, '월':4, '목':7}
    wc = week_list[str(dt.weekday())]
    week = week_code[wc]
    hour = dt.hour
    return day, week, hour

## 데이터 프레임형태로 만드는 함수
def Make_DataFrame(Departure_or_entry,Date):
    day, week, hour = Date_data(Date)
    Flight_time_hour = Flight_Time(Departure_or_entry,Class,Flight_type)
    return pd.DataFrame({'class': Class, 
                         'port_a': Port_a, 
                         'interior_airlines': Interior_airlines, 
                         'flight_type': Flight_type,
                         'flight_time_hour': Flight_time_hour, 
                         'port_d': Port_d,
                         'departure_hour': hour, 
                         'departure_date_day': day, 
                         'departure_day': week})


In [1]:
# 입력 데이터 (입력값 : 치환값)
## 출국 데이터 입력
Departure_or_entry = '출국' #비행기 방향(KA->LA 출국)
Class = input() #클래스 선택( 이코노미 : 1 / 비즈니스 : 2 / 프리미엄 이코노미 : 3 / 퍼스트 : 4 )
Port_d = input() #탑승 공항( K->L : ICN = 1 / GMP = 2 | L->K LAX = 1 / BUR = 2 )
Date = input() #날짜(-> departure_day 요일/departure_date_day 일/ departure_hour 시간/ departure_minute 분 으로 데이터가 나누어짐)
Port_a = input() #하차 공항( K->L : LAX = 1 / BUR = 2 | L->K GMP = 1 / ICN = 2 )
Interior_airlines = input() #국내 항공 선택(국내항공 선택 : 1 / 국내항공 선택 안함 : 0)
Flight_type = input() #직항 경유 선택 여부(직항 : 0 / 경유 회수 : 1, 2, 3, 4)
departure_df = Make_DataFrame(Departure_or_entry,Date)

## 귀국 데이터 입력
Departure_or_entry = '귀국' #비행기 방향(LA->KA 입국)
Class = input() #클래스 선택( 이코노미 : 1 / 비즈니스 : 2 / 프리미엄 이코노미 : 3 / 퍼스트 : 4 )
Port_d = input() #탑승 공항( K->L : ICN = 1 / GMP = 2 | L->K LAX = 1 / BUR = 2 )
Date = input() #날짜(-> departure_day 요일/departure_date_day 일/ departure_hour 시간/ departure_minute 분 으로 데이터가 나누어짐)
Port_a = input() #하차 공항( K->L : LAX = 1 / BUR = 2 | L->K GMP = 1 / ICN = 2 )
Interior_airlines = input() #국내 항공 선택(국내항공 선택 : 1 / 국내항공 선택 안함 : 0)
Flight_type = input() #직항 경유 선택 여부(직항 : 0 / 경유 회수 : 1, 2, 3, 4)
entry_df = Make_DataFrame(Departure_or_entry,Date)

In [ ]:
# 출국(KA -> LA)
with open('departure.pkl','rb') as pickle_file:
    model_departure = pickle.load(pickle_file)
departure_pred = model_departure.predict(departure_df)[0]

In [ ]:
# 귀국(LA -> KA)
with open('entry.pkl','rb') as pickle_file:
    model_entry = pickle.load(pickle_file)
entry_pred = model_departure.predict(entry_df)[0]

In [ ]:
# 티켓 완복 가격
Price = round(departure_pred + entry_pred,0)
print('출국 티켓값 : ',int(departure_pred))
print('입국 티켓값 : ',int(entry_pred))
print('총 티켓값 : ', int(Price))

In [12]:
test = datetime(2020, 12, 14, 14, 10, 50)
diff_days = timedelta(hours=30,minutes=20)
test2 = test + diff_days
test2.date()

datetime.date(2020, 12, 15)

In [34]:
# 도착 시간 출력 -> 수정 중...
def Arrival(Date,Flight_time_hour):
    departure = Date
    hour = int(Flight_time_hour)
    mminut = round(Flight_time_hour - hour,2)
    week_list = {'0':'월','1':'화','2':'수','3':'목','4':'금','5':'토','6':'일'}
    arrival = timedelta(hours=hour,minutes=mminut)
    wc = week_list[str(arrival.weekday())]
    return arrival.strftime('%y-%m-%d %H:%M'), wc


a, b = Arrival(datetime(2020, 12, 14, 14, 10, 50),11.34)
print(a,b)

AttributeError: 'datetime.timedelta' object has no attribute 'weekday'